# SBUS Demo

> Source data: https://ichoosr.egnyte.com/navigate/folder/701cce7f-a797-47c6-bcae-7ee2e3242e8c
> 
> Confluence: https://ichoosr.atlassian.net/wiki/x/IYBwMQE?atlOrigin=eyJpIjoiMzg2YWQxZmI2NjUyNDg1Y2E2YjMyZTViOGRjOWFmOGEiLCJwIjoiYyJ9

In [ ]:
%reload_ext setup

## Demo

In [ ]:
from igent.workflow import run_workflow

line = "sbus"
await run_workflow(
    model="azure", # "openai" or "azure"
    business_line=line,
    registrations_file=f"../data/demo/{line}_mock_registrations.json",
    offers_file=f"../data/demo/{line}_mock_offers.json",
    # incentives_file="../data/demo/incentives.json",
    matches_file=f"../data/demo/{line}_azure_matches.json",
    pos_file=f"../data/demo/{line}_azure_pos.json",
    max_items=1,
)